In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

C:\Users\sriha\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\ops\random_ops.py:285: SyntaxWarning: "is" with a literal. Did you mean "=="?
  minval_is_zero = minval is 0  # pylint: disable=literal-comparison
C:\Users\sriha\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\ops\random_ops.py:286: SyntaxWarning: "is" with a literal. Did you mean "=="?
  maxval_is_one = maxval is 1  # pylint: disable=literal-comparison
C:\Users\sriha\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\ops\ragged\ragged_batch_gather_with_default_op.py:84: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (default_value.shape.ndims is not 0
C:\Users\sriha\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\ops\ragged\ragged_batch_gather_with_default_op.py:85: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and default_value.shape.ndims is not 1):


In [2]:
file = open("metamorphosis_clean.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
print("The First Line: ", lines[0])
print("The Last Line: ", lines[-1])

The First Line:  One morning, when Gregor Samsa woke from troubled dreams, he found

The Last Line:  first to get up and stretch out her young body.


In [3]:
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
data[:360]


'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.  He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections.  The bedding was hardly able to cover it and seemed ready to slide off any moment.'

In [4]:
import string

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
new_data = data.translate(translator)

new_data[:500]

'One morning  when Gregor Samsa woke from troubled dreams  he found himself transformed in his bed into a horrible vermin   He lay on his armour like back  and if he lifted his head a little he could see his brown belly  slightly domed and divided by arches into stiff sections   The bedding was hardly able to cover it and seemed ready to slide off any moment   His many legs  pitifully thin compared with the size of the rest of him  waved about helplessly as he looked    What s happened to me   he'

In [5]:
z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)
data[:500]

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on armour-like back, and if lifted head little could see brown belly, slightly domed divided by arches stiff sections. The bedding was hardly able to cover it seemed ready slide off any moment. His many legs, pitifully thin compared with the size of rest him, waved about helplessly as looked. "What\'s happened me?" thought. It wasn\'t dream. room, proper human room altho'

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[17, 53, 293, 2, 18, 729, 135, 730, 294, 8]

In [7]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

2617


In [8]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]


The Length of sequences are:  3889


array([[ 17,  53],
       [ 53, 293],
       [293,   2],
       [  2,  18],
       [ 18, 729],
       [729, 135],
       [135, 730],
       [730, 294],
       [294,   8],
       [  8, 731]])

In [9]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

In [10]:
print("The Data is: ", X[:5])
print("The responses are: ", y[:5])

The Data is:  [ 17  53 293   2  18]
The responses are:  [ 53 293   2  18 729]


In [11]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             26170     
_________________________________________________________________
lstm (LSTM)                  (None, 1, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 2617)              2619617   
Total params: 15,694,787
Trainable params: 15,694,787
Non-trainable params: 0
_________________________________________________________________


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [16]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))

In [17]:
model.fit(X, y, epochs=200, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/150
61/61 [==============================] - ETA: 0s - loss: 7.8753
Epoch 00001: loss improved from inf to 7.87528, saving model to nextword1.h5
61/61 [==============================] - 9s 145ms/step - loss: 7.8753 - lr: 0.0010
Epoch 2/150
61/61 [==============================] - ETA: 0s - loss: 7.8647
Epoch 00002: loss improved from 7.87528 to 7.86472, saving model to nextword1.h5
61/61 [==============================] - 8s 136ms/step - loss: 7.8647 - lr: 0.0010
Epoch 3/150
61/61 [==============================] - ETA: 0s - loss: 7.8222
Epoch 00003: loss improved from 7.86472 to 7.82219, saving model to nextword1.h5
61/61 [==============================] - 8s 136ms/step - loss: 7.8222 - lr: 0.0010
Epoch 4/150
61/61 [==============================] - ETA: 0s - loss: 7.6889
Epoch 00004: loss improved from 7.82219 to 7.68889, saving model to nextword1.h5
61/61 [==============================] - 8s 136ms/step - loss: 7.6889 - lr: 0.0010
Epoch 5/150
61/61 [=========================

61/61 [==============================] - ETA: 0s - loss: 3.9174
Epoch 00035: loss improved from 3.99698 to 3.91740, saving model to nextword1.h5
61/61 [==============================] - 8s 133ms/step - loss: 3.9174 - lr: 0.0010
Epoch 36/150
61/61 [==============================] - ETA: 0s - loss: 3.8297
Epoch 00036: loss improved from 3.91740 to 3.82966, saving model to nextword1.h5
61/61 [==============================] - 8s 133ms/step - loss: 3.8297 - lr: 0.0010
Epoch 37/150
61/61 [==============================] - ETA: 0s - loss: 3.7528
Epoch 00037: loss improved from 3.82966 to 3.75279, saving model to nextword1.h5
61/61 [==============================] - 8s 133ms/step - loss: 3.7528 - lr: 0.0010
Epoch 38/150
61/61 [==============================] - ETA: 0s - loss: 3.6776
Epoch 00038: loss improved from 3.75279 to 3.67762, saving model to nextword1.h5
61/61 [==============================] - 8s 133ms/step - loss: 3.6776 - lr: 0.0010
Epoch 39/150
61/61 [=============================

61/61 [==============================] - ETA: 0s - loss: 1.6724
Epoch 00069: loss improved from 1.71207 to 1.67242, saving model to nextword1.h5
61/61 [==============================] - 8s 133ms/step - loss: 1.6724 - lr: 0.0010
Epoch 70/150
61/61 [==============================] - ETA: 0s - loss: 1.6483
Epoch 00070: loss improved from 1.67242 to 1.64829, saving model to nextword1.h5
61/61 [==============================] - 8s 136ms/step - loss: 1.6483 - lr: 0.0010
Epoch 71/150
61/61 [==============================] - ETA: 0s - loss: 1.6030
Epoch 00071: loss improved from 1.64829 to 1.60302, saving model to nextword1.h5
61/61 [==============================] - 8s 132ms/step - loss: 1.6030 - lr: 0.0010
Epoch 72/150
61/61 [==============================] - ETA: 0s - loss: 1.5582
Epoch 00072: loss improved from 1.60302 to 1.55822, saving model to nextword1.h5
61/61 [==============================] - 8s 134ms/step - loss: 1.5582 - lr: 0.0010
Epoch 73/150
61/61 [=============================

Epoch 104/150
61/61 [==============================] - ETA: 0s - loss: 1.0827
Epoch 00104: loss did not improve from 1.05690
61/61 [==============================] - 8s 134ms/step - loss: 1.0827 - lr: 0.0010
Epoch 105/150
61/61 [==============================] - ETA: 0s - loss: 1.0775
Epoch 00105: loss did not improve from 1.05690

Epoch 00105: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
61/61 [==============================] - 8s 131ms/step - loss: 1.0775 - lr: 0.0010
Epoch 106/150
61/61 [==============================] - ETA: 0s - loss: 0.8139
Epoch 00106: loss improved from 1.05690 to 0.81386, saving model to nextword1.h5
61/61 [==============================] - 8s 137ms/step - loss: 0.8139 - lr: 2.0000e-04
Epoch 107/150
61/61 [==============================] - ETA: 0s - loss: 0.7371
Epoch 00107: loss improved from 0.81386 to 0.73706, saving model to nextword1.h5
61/61 [==============================] - 8s 136ms/step - loss: 0.7371 - lr: 2.0000e-04
Epoch 108/

Epoch 139/150
61/61 [==============================] - ETA: 0s - loss: 0.6322
Epoch 00139: loss improved from 0.63228 to 0.63222, saving model to nextword1.h5
61/61 [==============================] - 8s 135ms/step - loss: 0.6322 - lr: 1.0000e-04
Epoch 140/150
61/61 [==============================] - ETA: 0s - loss: 0.6319
Epoch 00140: loss improved from 0.63222 to 0.63190, saving model to nextword1.h5
61/61 [==============================] - 8s 135ms/step - loss: 0.6319 - lr: 1.0000e-04
Epoch 141/150
61/61 [==============================] - ETA: 0s - loss: 0.6342
Epoch 00141: loss did not improve from 0.63190
61/61 [==============================] - 8s 134ms/step - loss: 0.6342 - lr: 1.0000e-04
Epoch 142/150
61/61 [==============================] - ETA: 0s - loss: 0.6315
Epoch 00142: loss improved from 0.63190 to 0.63151, saving model to nextword1.h5
61/61 [==============================] - 8s 135ms/step - loss: 0.6315 - lr: 1.0000e-04
Epoch 143/150
61/61 [=============================